## Sweep bot

i'm fucking crashing out

In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [3]:

import numpy as np
from pydrake.all import (
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [4]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [20]:
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    # we add diagram after building
    # really scuffed but i don't care at this point
    meta_controller_sys = MetaController(plant, station)
    meta_controller = builder.AddSystem(meta_controller_sys)

    # ID CONTROLLER
    _, iiwa_plant, _ = build_temp_plant()
    ID_controller = builder.AddSystem(
        InverseDynamicsController(
            iiwa_plant,
            kp = np.ones(7) * 200,
            kd = np.ones(7) * 40,
            ki = np.zeros(7),
            has_reference_acceleration=False,
        )
    )

    control_estimated_state_input_port = ID_controller.get_input_port(0)
    control_desired_state_input_port = ID_controller.get_input_port(1)
    control_output_port = ID_controller.get_output_port(0)

    demux = builder.AddSystem(Demultiplexer([14, 1]))

    builder.Connect(
        meta_controller.get_output_port(),
        demux.get_input_port(),
    )

    builder.Connect(
        demux.get_output_port(0),
        control_desired_state_input_port
    )

    builder.Connect(
        demux.get_output_port(1), 
        station.GetInputPort("wsg.position")
    )

    builder.Connect(
        station.GetOutputPort("iiwa_state"),
        control_estimated_state_input_port
    )

    builder.Connect(
        control_output_port,
        station.GetInputPort("iiwa_actuation")
    )

    diagram = builder.Build()
    meta_controller_sys.add_diagram(diagram)

    return diagram, plant, station

# diagram = builder.Build()

In [21]:
diagram, plant, station = make_diagram()

[[-0.10006814 -0.09808337 -0.09609859 ... -0.76686585 -0.7668732
  -0.76688063]
 [ 1.0237491   1.0237491   1.0237491  ...  0.61499244  0.61510605
   0.6152213 ]
 [ 0.0499999   0.0499999   0.0499999  ...  0.0417598   0.03298612
   0.0240788 ]]


TypeError: FirstOrderHold(): incompatible function arguments. The following argument types are supported:
    1. (breaks: list[float], samples: list[list[float]]) -> pydrake.trajectories.PiecewisePolynomial
    2. (breaks: list[float], samples: list[numpy.ndarray[numpy.float64[m, n]]]) -> pydrake.trajectories.PiecewisePolynomial

Invoked with: [0.0, 1.0], array([0, 0])

In [ ]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WPregrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.7, 0.5])
X_WGrasp = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.5])
X_WLift = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.60, 0.7])
AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrasp)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrasp)

for pos in [X_WPregrasp, X_WGrasp, X_WLift]:
    print(solve_ik_for_pose(plant, pos))
# AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
# AddMeshcatTriad(meshcat, 'b', X_PT=X_WGoal)


traj, wsg_traj = grasp_path(X_WStart, X_WPregrasp, X_WGrasp, X_WLift)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

In [ ]:
from scripts.broom_utils import get_broom_pregrip, get_broom_grip,compute_broom_grasp_angle
# from scripts.meta_controller import pregrip_to_grip_trajectory

plant_context = plant.CreateDefaultContext()
base = plant.GetBodyByName("base", plant.GetModelInstanceByName('iiwa'))
body = plant.GetBodyByName("handle_link")
X_WBroom = plant.EvalBodyPoseInWorld(plant_context, body)
X_WBase = plant.EvalBodyPoseInWorld(plant_context, base)
ang = compute_broom_grasp_angle(X_WBroom, X_WBase.translation())
print(ang)
X_WPregrip = get_broom_pregrip(X_WBroom, ang)
# X_WPregrip = X_WGoal
X_WGrip = get_broom_grip(X_WBroom, ang)
# AddMeshcatTriad(meshcat, 'broom', X_PT=X_WBroom)
# AddMeshcatTriad(meshcat, 'base', X_PT=X_WBase)
# AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrip)
import random
for i in range(1):
    x = random.uniform(-1, 1)
    y = random.uniform(1.6, 2)
    trans = RigidTransform(RotationMatrix(), [x, y, 0])
    ang = compute_broom_grasp_angle(trans, X_WBase.translation())
    grip = get_broom_grip(trans, ang)
    AddMeshcatTriad(meshcat, f'{i}', X_PT=grip)
AddMeshcatTriad(meshcat, 'pregrip', X_PT=X_WPregrip)
AddMeshcatTriad(meshcat, 'grip', X_PT=X_WGrip)


In [ ]:


traj_V_G, traj_wsg = plan_path(X_WStart, X_WPregrip)
builder, plant, station = make_builder()
add_controller(builder, station, traj_V_G, traj_wsg)
traj = traj_V_G
# traj2_V_G, traj2_wsg_command = pregrip_to_grip_trajectory(X_WPregrip, X_WGrip)

# traj_V_G = combine_trajectory(traj1_V_G, traj2_V_G)
# traj_wsg = combine_trajectory(traj1_wsg, traj2_wsg)

# grip the broom

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
# q_src = builder.AddSystem(TrajectorySource(traj))
# builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
# wsg_src = builder.AddSystem(TrajectorySource(traj_wsg))
# builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()
X_WPregrip


In [ ]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()

In [ ]:
from manipulation.utils import RenderDiagram
MetaController(plant, station, diagram, None, None, None, None)